# Исследование препроцессинга на распознавание номера документов прототипа OCR. Детекция OpenVINO. Распознавание Tesseract 
- Автор: Кирилл Киселев
- Дата начала: 13.05.2022
- Описание исследования: в датасете содержится 203 изображения. Сперва необходимо определить количество отфильтрованных изображений, затем получить результаты распознавания. Сделать выводы.

## Загрузка библиотек

In [151]:
import os
import pandas as pd
import cv2
import numpy as np

## Глобальные переменные

In [152]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks/ocr


In [153]:
TXT_PATH = '/home/kirillk/PycharmProjects/ocr_prototype_tesseract/ocr-prototype/assets/results/ocr_ovino_tesseract_iter_8.txt'

SAVE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results'
DATA_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/'

## Загрузка данных и подготовка датасета

In [154]:
filename_list = []
results_list = []

with open(TXT_PATH, "r") as file:
    for num, line in enumerate(file):
        if 'jpg' in line:
            line = line.split(']')[-1].rstrip()
            filename_list.append(line)
        elif 'FPS' in line:
            break
        else:
            results_list.append(line.rstrip())
            
# results_list = results_list[0:-1]

In [155]:
filename_list

['148.jpg',
 '083.jpg',
 '076.jpg',
 '146.jpg',
 '079.jpg',
 '065.jpg',
 '069.jpg',
 '165.jpg',
 '113.jpg',
 '180.jpg',
 '119.jpg',
 '081.jpg',
 '167.jpg']

In [156]:
len(filename_list)

13

In [157]:
results_list

["{'id': 0, 'barcode': '2860602715096', 'doc_type': 'Товарная ‚накладная', 'doc_number': None, 'error_list': ['Пересканируйте, текст не найден'], 'barcode_region': [163, 0, 1618, 5898], 'type_region': [1120, 624, 1877, 996], 'number_region': [1738, 636, 2201, 972]}",
 "{'id': 1, 'barcode': '2810096516426', 'doc_type': 'Товарная ‚накладная', 'doc_number': None, 'error_list': ['Пересканируйте, текст не найден'], 'barcode_region': [97, 0, 1266, 4133], 'type_region': [1496, 1325, 2595, 1697], 'number_region': [2379, 1341, 3109, 1680]}",
 "{'id': 2, 'barcode': '2810096516365', 'doc_type': 'Товарная накладная', 'doc_number': ')JOKYMeHTa', 'error_list': ['Пересканируйте, текст не найден'], 'barcode_region': [97, 0, 1266, 4133], 'type_region': [1524, 1310, 2623, 1682], 'number_region': [2407, 1326, 3137, 1665]}",
 "{'id': 3, 'barcode': '2860601473409', 'doc_type': None, 'doc_number': 'TE01T418140.', 'error_list': ['Пересканируйте, текст не найден'], 'barcode_region': [164, 0, 1626, 5910], 'typ

In [158]:
len(results_list)

13

In [159]:
df = pd.DataFrame()

for num, _ in enumerate(results_list):
    q = eval(results_list[num].replace('\\', ''))
    if q['error_list'] == []:
        q['error_list'] = 0
    elif len(q['error_list']) == 2:
        q['error_list'] = 2
    else:
        q['error_list'] = 1
        
    q['barcode_region'] = 0
    q['type_region'] = 0
    q['number_region'] =0
    
    res_df = pd.DataFrame(q, index=[0])
    df = pd.concat([df, res_df])

In [160]:
df = df.reset_index(drop=True)

In [161]:
df['filename'] = filename_list

In [162]:
df = df[['id', 'filename', 'barcode', 'doc_type', 'doc_number', 'error_list']]

In [163]:
df.head()

,id,filename,barcode,doc_type,doc_number,error_list
0,0,148.jpg,2860602715096,Товарная ‚накладная,None,1
1,1,083.jpg,2810096516426,Товарная ‚накладная,None,1
2,2,076.jpg,2810096516365,Товарная накладная,)JOKYMeHTa,1
3,3,146.jpg,2860601473409,None,TE01T418140.,1
4,4,079.jpg,2810097409826,Товарная ‚накладная,)JOKyMeHTa,1


## Визуализация

## Анализ данных

### Необходимые функции

In [164]:
def not_numeric_in_colunm(col):
    not_numeric = {}
    for i, value in enumerate(col):
        if col[i].isnumeric() == False:
            not_numeric[i] = value
    return not_numeric

In [165]:
def len_of_numbers(col):
    length_dict = {}
    for i, value in enumerate(col):
        if value.isnumeric():
            if len(value) not in length_dict.keys():
                length_dict[len(value)] = 1
            else:
                length_dict[len(value)] += 1
    return length_dict

### Doc number

In [166]:
df['doc_number'].isna().sum()

3

In [167]:
df['doc_number'] = df['doc_number'].fillna('None')

In [168]:
bad_doc_number = df[df['doc_number'] == 'None']
bad_doc_number.head(15)

,id,filename,barcode,doc_type,doc_number,error_list
0,0,148.jpg,2860602715096,Товарная ‚накладная,None,1
1,1,083.jpg,2810096516426,Товарная ‚накладная,None,1
8,8,113.jpg,2860597420210,Товарная 1накладная,None,1


In [169]:
bad_doc_number_df = df[df['doc_number'].str.isnumeric() == False]
display(bad_doc_number_df.tail(20))
print(len(bad_doc_number_df))

,id,filename,barcode,doc_type,doc_number,error_list
0,0,148.jpg,2860602715096,Товарная ‚накладная,None,1
1,1,083.jpg,2810096516426,Товарная ‚накладная,None,1
2,2,076.jpg,2810096516365,Товарная накладная,)JOKYMeHTa,1
3,3,146.jpg,2860601473409,None,TE01T418140.,1
4,4,079.jpg,2810097409826,Товарная ‚накладная,)JOKyMeHTa,1
5,5,065.jpg,2810097409826,Товарная накладная,)JOKYMEHTa,1
6,6,069.jpg,2810096516365,Товарная накладная,)AOKYMEHTa,1
7,7,165.jpg,2860611233796,None,[OOKymeHTa,1
8,8,113.jpg,2860597420210,Товарная 1накладная,None,1
9,9,180.jpg,2860618473492,Товарная |накладная,PAOKYMeHTA,1


12


In [170]:
len_not_match = []

for num, value in enumerate(df['doc_number']):
    if len(value) != 10:
        len_not_match.append(df['filename'][num])

In [171]:
len_not_match

['148.jpg', '083.jpg', '146.jpg', '113.jpg']

In [130]:
len(len_not_match)

5

In [131]:
good_doc_number_df = df[df['doc_number'].str.isnumeric() == True]
good_doc_number_df

,id,filename,barcode,doc_type,doc_number,error_list


In [132]:
bad_doc_number_filenames = set(list(bad_doc_number_df['filename']) + len_not_match)

In [133]:
len(bad_doc_number_filenames)

13

In [134]:
len(df) - len(bad_doc_number_filenames)

0

In [135]:
1 - len(bad_doc_number_filenames)/len(df)

0.0

In [136]:
bad_doc_number_filenames

{'065.jpg',
 '069.jpg',
 '076.jpg',
 '079.jpg',
 '081.jpg',
 '083.jpg',
 '113.jpg',
 '119.jpg',
 '146.jpg',
 '148.jpg',
 '165.jpg',
 '167.jpg',
 '180.jpg'}

In [137]:
len(bad_doc_number_filenames)

13

In [109]:
bad_doc_number_it_4 = ['008.jpg', '019.jpg', '038.jpg', '039.jpg', '041.jpg', '044.jpg', '047.jpg',
                       '052.jpg', '057.jpg', '065.jpg', '066.jpg', '068.jpg', '069.jpg', '074.jpg',
                       '076.jpg', '079.jpg', '080.jpg', '081.jpg', '082.jpg', '083.jpg', '084.jpg',
                       '091.jpg', '095.jpg', '099.jpg', '103.jpg', '105.jpg', '109.jpg', '113.jpg',
                       '116.jpg', '123.jpg', '125.jpg', '127.jpg', '128.jpg', '129.jpg', '130.jpg',
                       '131.jpg', '132.jpg', '136.jpg', '138.jpg', '141.jpg', '144.jpg', '145.jpg',
                       '146.jpg', '148.jpg', '150.jpg', '151.jpg', '158.jpg', '162.jpg', '164.jpg', 
                       '165.jpg', '167.jpg', '175.jpg', '176.jpg', '179.jpg', '180.jpg', '182.jpg',
                       '183.jpg', '184.jpg', '188.jpg', '189.jpg', '190.jpg', '191.jpg', '193.jpg', 
                       '194.jpg', '200.jpg', '201.jpg', '202.jpg']

In [416]:
bad_doc_number_it_5 = ['065.jpg', '069.jpg', '076.jpg', '079.jpg', '081.jpg', '083.jpg', '109.jpg', '113.jpg',
                       '116.jpg', '119.jpg', '123.jpg', '128.jpg', '129.jpg', '136.jpg', '141.jpg', '145.jpg',
                       '146.jpg', '148.jpg', '162.jpg', '165.jpg', '167.jpg', '169.jpg', '180.jpg', '183.jpg',
                       '184.jpg', '189.jpg', '190.jpg']

In [46]:
bad_doc_number_it_6 = ['065.jpg', '069.jpg', '076.jpg', '079.jpg', '081.jpg', '083.jpg', '113.jpg', '119.jpg',
                       '123.jpg', '128.jpg', '136.jpg', '141.jpg', '146.jpg', '148.jpg', '162.jpg', '165.jpg', 
                       '167.jpg', '180.jpg', '189.jpg']

In [77]:
bad_doc_number_it_7 = ['065.jpg', '069.jpg', '076.jpg', '079.jpg', '081.jpg', '083.jpg', '113.jpg', '119.jpg', 
                       '146.jpg', '148.jpg', '165.jpg', '167.jpg', '180.jpg']

In [78]:
len(bad_doc_number_it_7)

13

In [79]:
good_doc_numbers = df[~df['filename'].isin(bad_doc_number_it_7)]['doc_number']
good_doc_numbers

0     6017853136
11    8017407243
12    6017080275
16    8017085704
17    6017080275
18    6017266228
Name: doc_number, dtype: object

In [80]:
good_doc_numbers.str.startswith('6').value_counts()

True     4
False    2
Name: doc_number, dtype: int64

1. iter_6: third+(убрал outputbase digits) - 19 нераспознанных изображений
1. iter_4: fourth+(убрал outputbase digits) - 13 нераспознанных изображений

#### Вывод 
В столбце с номерами документов 27 пропущенных значения. В них фигурируют либо **None** либо нечисловые. Качественно распознано 166 номеров документов из 193 примеров, или всего 86%.

## Результат исследования

In [412]:
bad_lists_union = sorted(list(set().union(bad_doc_type_list, bad_doc_number_filenames)))

In [413]:
len(bad_lists_union)

28

In [414]:
len(good_df) - len(bad_lists_union)

165

In [415]:
1 -len(bad_lists_union) / len(good_df)

0.8549222797927462

## Общий вывод

**Итерация 1** - отсутствует предобработка (первый препроцессинг). Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

**Итерация 2** - добавлен второй препроцессинг. Можно считать качественно распознанными 178 документов из 193, что составляет **92.70%** из тестовых данных.

**Итерация 3** - добавлен третий препроцессинг. Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

**Итерация 4** - добавлен четвертый препроцессинг. Можно считать качественно распознанными 180 документов из 193, что составляет **93.26%** из тестовых данных.

**Итерация 5** - добавлен пятый препроцессинг. Можно считать качественно распознанными 180 документов из 193, что составляет **93.26%** из тестовых данных.

**Итерация 6** - добавлен новый препроцессинг. 

`return cv2.GaussianBlur(img, (1, 1), 0)`

Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

In [150]:
bad_lists_union

['038.jpg',
 '039.jpg',
 '041.jpg',
 '044.jpg',
 '047.jpg',
 '057.jpg',
 '079.jpg',
 '082.jpg',
 '091.jpg',
 '095.jpg',
 '096.jpg',
 '099.jpg',
 '109.jpg',
 '113.jpg',
 '116.jpg',
 '119.jpg',
 '125.jpg',
 '127.jpg',
 '131.jpg',
 '132.jpg',
 '136.jpg',
 '138.jpg',
 '141.jpg',
 '144.jpg',
 '145.jpg',
 '146.jpg',
 '148.jpg',
 '150.jpg',
 '158.jpg',
 '164.jpg',
 '165.jpg',
 '167.jpg',
 '169.jpg',
 '175.jpg',
 '177.jpg',
 '179.jpg',
 '180.jpg',
 '182.jpg',
 '184.jpg',
 '188.jpg',
 '190.jpg',
 '193.jpg',
 '201.jpg',
 '202.jpg']

In [81]:
bad_doc_number_it_7 = ['065.jpg', '069.jpg', '076.jpg', '079.jpg', '081.jpg', '083.jpg', '113.jpg', '119.jpg', 
                       '146.jpg', '148.jpg', '165.jpg', '167.jpg', '180.jpg']

In [139]:
bad_doc_number_it_7 = ['165.jpg', '167.jpg', '180.jpg']

In [182]:
for i in bad_doc_number_it_7:
    image = cv2.imread(DATA_PATH + i)
    kernel = np.ones((2, 2), 'uint8')
    
#     image = cv2.erode(image, kernel, iterations=2)
#     image = cv2.GaussianBlur(image, (3, 3), 1)
    image = cv2.dilate(image, kernel, iterations=2)
    
    kernel = np.ones((1, 1), 'uint8')
    image = cv2.erode(image, kernel, iterations=3)
    image = cv2.GaussianBlur(image, (5, 5), 4)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(image, 210, 255, cv2.THRESH_BINARY)  # 130
    image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)
    cv2.imshow(f'Image {i}', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [455]:
for i in bad_doc_number_it_7:
    image = cv2.imread(DATA_PATH + i)
    kernel = np.ones((1, 1), 'uint8')
    
    image = cv2.GaussianBlur(image, (7, 7), 1)
#     image = cv2.dilate(image, kernel, iterations=2)
#     image = cv2.GaussianBlur(image, (5, 5), 4)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     _, image = cv2.threshold(image, 140, 255, cv2.THRESH_BINARY)  # 130
    image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)
    cv2.imshow(f'Image {i}', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Notes